# linear regression
## spring
[link](https://docs.google.com/presentation/d/1L1LwpKm5DxhHndiyyiZ3wJA2mKOJTQ2heKo45Me5yVg/edit#slide=id.g1eaee6a347_0_8)

## fall
[link](https://ntumlta.github.io/2017fall-ml-hw1/)

- hw1.sh
 - Python3.5+ required
 - Only (1)numpy (2)scipy (3)pandas are allowed
 - numpy.linalg.lstsq is forbidden.
 - Please handcraft "linear regression" using Gradient Descent
 - beat public simple baseline
 - For those who wish to load model instead of running whole training precess:
 - please upload your training code named train.py
 - as long as there are Gradient Descent Code in train.py, it's fine
- hw1_best.sh
 - Python3.5+ required
 - any library is allowed
 - meet the higher score you choose in kaggle
 
 ### Data 簡介

* [train.csv](./data/train.csv) : 每個月前20天每個小時的氣象資料(每小時有18種測資)。共12個月。
* [test.csv](./data/test.csv) : 排除train.csv中剩餘的資料，取連續9小時的資料當feature，預測第10小時的PM2.5值。總共取240筆不重複的test data。
* [sampleSubmission.csv](./data/sampleSubmission.csv)
* [testing答案](./data/ans.csv)

## linear regression

找出 $f$ 使得  
loss function() $$L(f) = \sum_{f} (\hat{y}^n - f(x))^2$$ 最小 $f^* = arg \displaystyle \min_{f} L(f)$  
又因為 $f(X) = b + w \cdot X$, where $X \in \Omega, b \in \Bbb{F}$
可以寫成 $$L(b, w) = \sum_{b, w} (\hat{y}^n - b - w \cdot X)^2$$
引此可以將題目變成找
$$b^*, w^* = arg \min_{b,w} \sum_{b, w} (\hat{y}^n - b - w \cdot X)^2$$

In [44]:
import numpy as np
import pandas as pd
train_data_file = "./data/train.csv"
df = pd.read_csv(train_data_file, parse_dates={'date':[0]}, encoding='big5')

In [71]:
df.head(18)

,date,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014-01-01,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014-01-01,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014-01-01,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014-01-01,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014-01-01,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014-01-01,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014-01-01,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014-01-01,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014-01-01,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014-01-01,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [80]:
df[df['測項']!='RAINFALL'].iloc[:,3:27]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,...,11,11,22,28,19,12,8.1,7,6.9,6
6,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,16,30,27,23,24,28,24,22,21,29,...,65,64,51,34,33,34,37,38,38,36
8,56,50,48,35,25,12,4,2,11,38,...,52,51,66,85,85,63,46,36,42,42
9,26,39,36,35,31,28,25,20,19,30,...,36,45,42,49,45,44,41,30,24,13


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn

0      2014-01-01
1      2014-01-01
2      2014-01-01
3      2014-01-01
4      2014-01-01
5      2014-01-01
6      2014-01-01
7      2014-01-01
8      2014-01-01
9      2014-01-01
10     2014-01-01
11     2014-01-01
12     2014-01-01
13     2014-01-01
14     2014-01-01
15     2014-01-01
16     2014-01-01
17     2014-01-01
18     2014-01-02
19     2014-01-02
20     2014-01-02
21     2014-01-02
22     2014-01-02
23     2014-01-02
24     2014-01-02
25     2014-01-02
26     2014-01-02
27     2014-01-02
28     2014-01-02
29     2014-01-02
          ...    
4290   2014-12-19
4291   2014-12-19
4292   2014-12-19
4293   2014-12-19
4294   2014-12-19
4295   2014-12-19
4296   2014-12-19
4297   2014-12-19
4298   2014-12-19
4299   2014-12-19
4300   2014-12-19
4301   2014-12-19
4302   2014-12-20
4303   2014-12-20
4304   2014-12-20
4305   2014-12-20
4306   2014-12-20
4307   2014-12-20
4308   2014-12-20
4309   2014-12-20
4310   2014-12-20
4311   2014-12-20
4312   2014-12-20
4313   2014-12-20
4314   201

,日期,測站,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
測項,,,,,,,,,,,,,,,,,,,,,
AMB_TEMP,2014/1/12014/1/22014/1/32014/1/42014/1/52014/1...,豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原...,1416181913131415141213171513111313139.61215182...,1415181812131415131213171414111313129.81214171...,1415181813131415131313171414111413121011141819...,1314181712141415131213161414111312121012141819...,1214181712141415121313161414111212139.71214181...,1215181612151415121312161414101212131011141818...,1216181712141415121312151313111212121010141718...,1216191713141416131312141313111213121010141819...,...,2224241921202022152019201713182020171918242526...,2224221820191920141919191513182020151818232525...,2123221819182019131819181513171919141717232425...,1921211617171918121718161412151817131715222323...,1720201616161818121517151412141616111515192222...,1619201516151917121416151412141515101514182222...,15182015151517161214161514121314159.9151419212...,15182015141516151214161514121214149.9151418202...,15182014141516141213161514121313149.6141317201...,15181913131416141213171514111313139.3131319201...
CH4,2014/1/12014/1/22014/1/32014/1/42014/1/52014/1...,豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原...,1.81.81.81.81.81.81.81.81.81.81.81.81.81.81.71...,1.81.81.81.81.91.81.81.81.81.81.81.81.81.81.71...,1.81.81.81.81.91.81.81.81.81.81.81.81.81.81.71...,1.81.81.81.81.81.81.81.81.81.81.81.81.81.81.71...,1.81.81.81.81.81.81.81.81.81.81.81.81.81.81.71...,1.81.81.71.81.81.81.81.81.81.81.81.81.81.81.81...,1.81.81.71.81.81.81.81.81.81.81.81.81.81.81.71...,1.81.81.71.81.81.81.81.81.81.81.81.81.81.81.71...,...,1.81.81.81.81.81.81.81.81.81.81.81.71.81.81.71...,1.81.81.81.81.81.81.71.71.81.81.81.71.81.81.71...,1.81.81.81.81.81.81.81.71.81.81.81.71.81.81.71...,1.81.81.81.81.81.91.81.81.81.81.81.71.81.81.71...,1.81.81.81.81.81.91.81.81.81.81.81.71.81.81.81...,1.81.81.81.81.81.91.81.81.81.81.81.71.81.81.71...,1.81.81.81.81.81.91.81.81.81.81.81.71.81.81.71...,1.81.81.81.81.81.81.81.81.81.81.91.71.81.81.81...,1.81.81.81.81.81.81.81.81.81.81.91.81.81.81.71...,1.81.81.81.91.81.81.81.81.81.81.81.81.81.81.71...
CO,2014/1/12014/1/22014/1/32014/1/42014/1/52014/1...,豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原...,0.510.260.320.350.590.430.320.40.490.280.410.5...,0.410.250.350.350.560.430.320.410.460.270.360....,0.390.280.370.320.540.40.30.430.410.260.340.43...,0.370.270.310.330.540.410.290.410.390.260.340....,0.350.240.290.320.540.410.30.360.380.250.340.3...,0.30.260.270.330.540.390.310.370.360.270.350.3...,0.370.340.30.350.540.40.370.460.360.30.380.360...,0.470.560.510.410.520.570.710.630.40.460.460.5...,...,0.370.340.310.480.470.530.310.350.40.30.30.240...,0.370.350.340.530.470.460.340.310.370.30.330.2...,0.470.380.420.590.470.550.430.340.360.320.370....,0.690.610.530.70.590.770.650.480.40.450.370.27...,0.560.440.550.660.50.880.580.390.350.590.440.2...,0.450.40.60.790.480.660.550.40.340.450.40.260....,0.380.40.490.640.50.690.490.410.310.360.570.26...,0.350.550.280.660.510.50.430.480.30.380.650.27...,0.360.410.350.680.440.410.420.550.290.420.770....,0.320.330.370.650.410.350.440.530.280.420.560....
NMHC,2014/1/12014/1/22014/1/32014/1/42014/1/52014/1...,豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原豐原...,0.20.060.120.240.220.090.070.150.090.050.170.1...,0.150.050.180.180.150.090.090.140.080.040.130....,0.130.060.150.530.250.130.060.130.080.060.120....,0.120.050.130.280.240.090.060.120.070.040.120....,0.110.050.160.240.210.110.070.10.070.040.120.1...,0.060.070.070.220.160.080.080.130.070.090.110....,0.10.090.070.090.10.090.20.170.080.070.120.130...,0.130.190.150.10.080.140.220.240.080.150.140.1...,...,0.10.120.180.180.080.270.140.160.150.110.090.0...,0.130.160.190.140.080.220.180.160.10.130.110.0...,0.140.230.220.230.080.320.30.110.130.180.160.0...,0.230.320.320.180.130.530.340.150.160.220.130....,0.180.180.240.190.10.50.250.120.190.290.160.08...,0.120.150.30.270.10.290.230.130.120.190.130.06...,0.10.230.230.150.10.30.190.160.070.150.230.060...,0.090.290.060.140.110.160.150.16

## gradient descent
$x_{n+1} = x_n - \eta_n\nabla{f(x_n)} $